# Purpose:

The goal of this script is to go through the audio file names and turn those strings into vectors. I then save these vectors in a dictionary with their file names as keys. 



In [3]:
import os
from my_text_process_scripts import *
import gensim
from gensim.models.keyedvectors import KeyedVectors
from gensim.models.word2vec import Word2Vec

data_folder="../data/"
sound_folder="../sound_effects/sounds/"
create_word_vectors=False

word_vectors=[]



In [4]:

if create_word_vectors:
    word_vectors = KeyedVectors.load_word2vec_format(os.path.join(data_folder,"GoogleNews-vectors-negative300.bin.gz"), unicode_errors='ignore',binary=True, limit=100000)  

    word_vectors.init_sims(replace=True)
    word_vectors.save_word2vec_format(os.path.join(data_folder,'GoogleNews-vectors-normed2.bin'),binary=True)
    
else:

    word_vectors =KeyedVectors.load_word2vec_format(os.path.join(data_folder,'GoogleNews-vectors-normed2.bin'),binary=True, encoding='utf-8', unicode_errors='ignore')
    


C:\Users\Alex\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:

word_vectors=KeyedVectors.load_word2vec_format(os.path.join(data_folder,'GoogleNews-vectors-normed2.bin'),encoding="ISO-8859-1",binary=True)

In [6]:

def wordlist2vec(words_list,word_vectors):
    final_results=[]
    for word in words_list:
        try:
            if len(final_results)==0:

                final_results=word_vectors[word]
            else:

                final_results=final_results+word_vectors[word]
        except:
            print("word not found: "+word)
            pass
        
    if len(final_results)==0:
        final_results=np.zeros(300)
        
    final_results=final_results/len(words_list)
    return final_results


# to get the cosine similarity...

def vecSim(vec1,vec2):
    answer=sklearn.metrics.pairwise.cosine_similarity(vec1.reshape(1,-1),vec2.reshape(1,-1))
    
    return answer[0][0]


## going through the audio files

In [7]:

wav_to_vec_dict={}
for sub_sound_folder in os.listdir(sound_folder):
    
    sub_sound_path=os.path.join(sound_folder,sub_sound_folder)
    for soundfile in os.listdir(sub_sound_path):
        
        #adding the folder name to the sound file name
        text=sub_sound_folder+" "+soundfile[:-3]
        
        
        tokenized_resulting_string=tokenize_process(text) #this splits up the word and takes care of basic preprocessing

        
        individual_word_tags=[]
        words_in_voc=[]
        overall_tags=[]
        
        #CHECKING FOR WHAT'S IN THE DICTIONARY
        for each_word in tokenized_resulting_string:

            if each_word in word_vectors.vocab:

                words_in_voc.append(each_word)
             
        if len(words_in_voc)!=0:

            wav_to_vec_dict[os.path.join(sub_sound_folder,soundfile)]=wordlist2vec(words_in_voc,word_vectors)
            
        else:
            
            print("None of these words in the dictionary")
            print(text)
            

    

In [ ]:
import pickle

def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

        
save_obj(wav_to_vec_dict,"wav_to_vec_dict")
